In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import datasets, transforms

device = torch.device("cuda")

In [ ]:
dataset = datasets.MNIST("data", train=True, download=True)

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [ ]:
image, label = dataset[1]
print(label)

0


In [ ]:
image

In [ ]:
transforms.functional.to_tensor(image).shape

torch.Size([1, 28, 28])

In [ ]:
transform = transforms.ToTensor()

train_dataset = datasets.MNIST("data", train=True, download=True, transform=transform)
test_dataset = datasets.MNIST("data", train=False, download=True, transform=transform)


In [ ]:
image, label = train_dataset[1]
image.shape


torch.Size([1, 28, 28])

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)


In [ ]:
for images_batch, labels_batch in train_loader:
    print(images_batch.shape)
    print(labels_batch.shape)
    break

torch.Size([64, 1, 28, 28])
torch.Size([64])


**EXTRA** How to split dataset in val and train

In [ ]:
# from torch.utils.data import random_split
# train_dataset, val_dataset = random_split(dataset, lengths=[int(0.75*len(dataset)), len(dataset)-int(0.75*len(dataset))])


### Exercise 1

For the time being, we will use a very simple network. It consists of a sequence of two `nn.Linear` layers, which are densely-connected (also called "fully-connected") neural layers. The last layer is a 10-way `nn.LogSoftmax` layer, which means it will return an array of 10 log-probability scores. Each score will be the probability that the current digit image belongs to one of our 10 digit classes. Please fill in the network definition below.

In [ ]:
# Define a variable 'network' by instantiating a PyTorch sequential model
# https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential

# Add a (1) nn.Linear hidden layer with 128 neurons,, (2) a nn.ReLU, (3) the output nn.Linear and,
# the (4) output nn.LogSoftmax
# images_batch.shape = [64, 1, 28, 28]

num_inputs = 28*28

#TODO
network = nn.Sequential(
    nn.Linear(num_inputs, 128),
    nn.ReLU(),
    nn.Linear(128, 10),
    nn.LogSoftmax(),
)

### Exercise 2

To make our network ready for training, we need to pick three more things:

*    **A loss function**: this is how the network will be able to measure how good a job works on its training data, and thus how it will be able to steer itself in the right direction. **Check the [PyTorch documentation](https://pytorch.org/docs/stable/nn.functional.html#nll-loss) for the negative log likelihood loss for multi-class classification.**
*   **An optimizer**: this is the mechanism through which the network will update itself based on the data it sees and its loss function. **Check the PyTorch documentation and find the RMSprop optimizer to use it.**





In [ ]:
# Import optmizer 
# https://pytorch.org/docs/stable/optim.html#torch.optim.RMSprop
import torch.optim as optim
import torch.nn as nn

# Optimizer: RMS Prop 
lr = 1e-3
optimizer = optim.RMSprop(network.parameters(), lr)

# Negative Log Likelihood (NLL) 
criterion = nn.NLLLoss()
# criterion(y, y_)

# F.nll_loss(y, y_)

def accuracy(probabilities, labels):
    # probabilities = [64, 10]
    pred = probabilities.argmax(dim=1)
    return (pred == labels).float().mean().item()

In [ ]:
a = torch.tensor([[1, 2, 1, 6, -2], [1, -1, 10, 6, 2]])
b = torch.tensor([5, 2])
a

tensor([[ 1,  2,  1,  6, -2],
        [ 1, -1, 10,  6,  2]])

In [ ]:
preds = a.argmax(dim=1)

In [ ]:
(b == preds).float().mean().item()

0.5

### Exercise 3

Next, the training and test functions that iterate for an epoch are defined. They are missing a key ingredient to make the `data` tensors, which include images, compatible with the fully connected input of size `num_inputs` of the network. Please fix this in both cases, remembering the use of `reshape()` in the previous lab.

In [ ]:
def train_epoch(train_loader, network, optimizer, criterion):

  # Activate the train=True flag inside the model
  network.train()
  
  # For each batch
  for batch_idx, (data, target) in enumerate(train_loader):
      data = data.to(device)
      target = target.to(device)
      optimizer.zero_grad()
      # TO DO
      data = torch.flatten(data, start_dim=1)
      output = network(data)
      loss = criterion(output, target)
      loss.backward()
      optimizer.step()
      if batch_idx % 50 == 0:
        print(f"TRAIN Loss: {loss.item():.2f}, Accuracy: {accuracy(output, target):.2f}")


In [ ]:
def test_epoch(test_loader, network, criterion):

    # Dectivate the train=True flag inside the model
    network.eval()
    
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            # Load data and feed it through the neural network
            data, target = data.to(device), target.to(device)
            data = data.view(data.shape[0], -1)
            output = network(data)

            loss = criterion(output, target)

            if batch_idx % 50 == 0:
                print(f"TEST Loss: {loss.item():.2f}, Accuracy: {accuracy(output, target):.2f}")

In [ ]:
data = torch.rand(64, 1, 28, 28)
print(data.shape)
print(data.view(data.shape[0], -1).shape)

print(data.reshape(data.shape[0], data.shape[2]*data.shape[3]).shape)

print(torch.flatten(data, start_dim=1).shape)


torch.Size([64, 1, 28, 28])
torch.Size([64, 784])
torch.Size([64, 784])
torch.Size([64, 784])


In [ ]:
epochs = 50
network = network.to(device)
for epoch in range(epochs):
    train_epoch(train_loader, network, optimizer, criterion)
    test_epoch(test_loader, network, criterion)


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


TRAIN Loss: 0.07, Accuracy: 1.00
TRAIN Loss: 0.22, Accuracy: 0.94
TRAIN Loss: 0.19, Accuracy: 0.95
TRAIN Loss: 0.11, Accuracy: 0.98
TRAIN Loss: 0.09, Accuracy: 0.97
TRAIN Loss: 0.06, Accuracy: 0.97
TRAIN Loss: 0.08, Accuracy: 1.00
TRAIN Loss: 0.18, Accuracy: 0.94
TRAIN Loss: 0.13, Accuracy: 0.94
TRAIN Loss: 0.16, Accuracy: 0.95
TRAIN Loss: 0.08, Accuracy: 0.97
TRAIN Loss: 0.22, Accuracy: 0.92
TRAIN Loss: 0.13, Accuracy: 0.98
TRAIN Loss: 0.04, Accuracy: 0.98
TRAIN Loss: 0.04, Accuracy: 1.00
TRAIN Loss: 0.05, Accuracy: 1.00
TRAIN Loss: 0.30, Accuracy: 0.94
TRAIN Loss: 0.01, Accuracy: 1.00
TRAIN Loss: 0.09, Accuracy: 0.97
TEST Loss: 0.08, Accuracy: 0.98
TEST Loss: 0.08, Accuracy: 0.98
TEST Loss: 0.03, Accuracy: 1.00
TEST Loss: 0.16, Accuracy: 0.95
TRAIN Loss: 0.12, Accuracy: 0.95
TRAIN Loss: 0.09, Accuracy: 0.97
TRAIN Loss: 0.09, Accuracy: 0.97
TRAIN Loss: 0.25, Accuracy: 0.95
TRAIN Loss: 0.03, Accuracy: 0.98
TRAIN Loss: 0.06, Accuracy: 1.00
TRAIN Loss: 0.17, Accuracy: 0.97
TRAIN Loss: 0.

KeyboardInterrupt: ignored